In [1]:
# Imports
from ctraptools.kymos.detect import Detector
from lumicks import pylake

import ctraptools.kymos.io as kio
import matplotlib.pyplot as plt
import numpy as np
import tifffile

In [2]:
# Parameters
filepath = "/Users/sc13967/Documents/People/Alex Hughes-Games/2023-12-11 Binding rebinding/EcoP15I sliding diffusion/"
filename = "20230512-150855 Kymograph 4 into 4 mM ATP sliding"
extension = "h5"
channel = 0

In [3]:
# Getting kymograph image
file = pylake.File(filepath+filename+"."+extension)
kymo = next(iter(file.kymos.values()))
image = kymo.get_image()[:,700:6000,channel]
tifffile.imwrite(
            filepath+filename+"_raw.tif", 
            kymo.get_image().astype(np.single)
        )


In [4]:
# Detecting tracks
detector = Detector(half_t_w=0,peak_det_thresh = 12, n_max=5, c_ub=6, c_def=4, max_dist=6,max_frame_gap=50, min_track_length = 100, min_track_density=0.5, track_heritage_weight=100)
tracks = detector.detect(image)

100%|██████████| 5300/5300 [00:15<00:00, 343.18it/s]


In [5]:
# Saving files
kio.write_peak_traces(tracks,filepath+filename+'_C'+str(channel)+"_traces")
kio.save_overlay(tracks,image,filepath+filename+'_C'+str(channel)+"_overlay")


In [6]:
# Estimating noise in position
track = next(tracks.values().__iter__())

x = track.get_positions()[0:1000]

x_std = np.std(x)

In [7]:
link_dist = 2

for track in tracks.values():
    counts = np.zeros(image.shape)

    for curr_timepoint, curr_peak in sorted(list(track.peaks.items()), reverse=True):
        curr_position = curr_peak.b

        # Comparing to all other peaks in this track
        for prev_timepoint, prev_peak in sorted(list(track.peaks.items()), reverse=True):
            prev_position = prev_peak.b

            # Only process peaks from earlir timepoints
            if prev_timepoint >= curr_timepoint:
                continue

            timepoint_separation = curr_timepoint - prev_timepoint
            position_separation = abs(curr_position-prev_position)

            if position_separation < link_dist:
                counts[round(curr_position),timepoint_separation] = counts[round(curr_position),timepoint_separation] + 1
            else:
                # Once the peak has moved out of the linking range, stop recording history
                break

    for f in range(image.shape[1]):
        for x in range(image.shape[0]):
            if counts[x,f] > 0:
                counts[x,f] = counts[x,f] / (max(track.peaks.keys())-min(track.peaks.keys())-f-1)
                # counts[x,f] = counts[x,f] / (image.shape[1]-f-1)

    tifffile.imwrite(
            filepath+filename+"_track"+str(track.ID)+"_counts.tif", 
            counts.astype(np.single)
        )
    